In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ast import literal_eval

In [2]:
# Data available at - https://www.kaggle.com/rounakbanik/the-movies-dataset

data = pd.read_csv('movies_metadata.csv')
data.head()

/home/parth/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
data['genres'] = data['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])    # converting genres to a list
data['genres'][0:10]

0           [Animation, Comedy, Family]
1          [Adventure, Fantasy, Family]
2                     [Romance, Comedy]
3              [Comedy, Drama, Romance]
4                              [Comedy]
5      [Action, Crime, Drama, Thriller]
6                     [Comedy, Romance]
7    [Action, Adventure, Drama, Family]
8         [Action, Adventure, Thriller]
9         [Adventure, Action, Thriller]
Name: genres, dtype: object

In [4]:
data['year'] = data['release_date'].apply(lambda x: str(x).split('-')[0])    # Clean up date
data['year'][0:10]

0    1995
1    1995
2    1995
3    1995
4    1995
5    1995
6    1995
7    1995
8    1995
9    1995
Name: year, dtype: object

In [5]:
"""
Weighted metric TMDB

= (votes/(votes+min votes)) avg rating + (votes/(votes+min votes)) Mean vote
"""

'\nWeighted metric TMDB\n\n= (votes/(votes+min votes)) avg rating + (votes/(votes+min votes)) Mean vote\n'

In [6]:
C = data['vote_average'].mean()
C

5.618207215134185

In [8]:
data['vote_count'] = data['vote_count'].fillna(data['vote_count'].mean())   
minvotes = data['vote_count'].astype(int).quantile(0.85)  # Minimum votes, play around with the number to see different results
minvotes

82.0

In [9]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+minvotes) * R) + (minvotes/(minvotes+v) * C)

In [10]:
qualified = data[(data['vote_count'] >= minvotes) & (data['vote_count'].notnull()) & (data['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(6832, 6)

In [11]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)
qualified['wr'][0:10]

0     6.979387
1     5.987452
2     5.820075
4     5.198796
5     6.942425
6     5.859610
8     5.198019
9     5.975465
10    5.888587
11    5.173606
Name: wr, dtype: float64

In [12]:
qualified = qualified.sort_values('wr', ascending=False).head(250)

In [13]:
s = data.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = data.drop('genres', axis=1).join(s)

<ipython-input-13-722603c64053>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = data.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)


In [14]:
def top_movies(genre, percentile = 0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

In [15]:
top_movies('Romance')

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457,8.565285
351,Forrest Gump,1994,8147,8,48.3072,7.971357
876,Vertigo,1958,1162,8,18.2082,7.811667
40251,Your Name.,2016,1030,8,34.461252,7.789489
883,Some Like It Hot,1959,835,8,11.8451,7.745154
...,...,...,...,...,...,...
7582,How to Steal a Million,1966,133,7,12.3896,6.352112
6912,Beauty and the Beast,1946,133,7,4.79885,6.352112
24888,Remember Sunday,2013,131,7,11.1906,6.346273
2801,Body Heat,1981,128,7,5.99148,6.337313


In [16]:
top_movies('Action')

,title,year,vote_count,vote_average,popularity,wr
15480,Inception,2010,14075,8,29.1081,7.955109
12481,The Dark Knight,2008,12269,8,123.167,7.948621
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.0707,7.929594
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.3244,7.924047
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.4235,7.918400
...,...,...,...,...,...,...
21296,The Wolverine,2013,4110,6,3.91829,5.956645
13635,X-Men Origins: Wolverine,2009,4086,6,1.45654,5.956404
5244,Star Wars: Episode II - Attack of the Clones,2002,4074,6,14.0725,5.956282
18289,Mission: Impossible - Ghost Protocol,2011,4026,6,14.2584,5.955789


In [18]:
top_movies('Animation')

,title,year,vote_count,vote_average,popularity,wr
359,The Lion King,1994,5520,8,21.6058,7.909446
5481,Spirited Away,2001,3968,8,41.0489,7.876077
9698,Howl's Moving Castle,2004,2049,8,16.136,7.772353
2884,Princess Mononoke,1997,2041,8,17.1667,7.771556
5833,My Neighbor Totoro,1988,1730,8,13.5073,7.735559
...,...,...,...,...,...,...
21990,Free Birds,2013,259,5,7.71323,5.416101
13296,The Tale of Despereaux,2008,260,5,12.3646,5.415274
19328,Cinderella II: Dreams Come True,2002,265,5,8.00271,5.411193
6020,The Jungle Book 2,2003,278,5,10.6059,5.400946
